In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("BatterySimV3.py"))
parent = os.path.dirname(os.path.dirname(current))
sys.path.append(parent+"\Functions")

#From Functions folder
from Merge import merge
from Battery import Battery
from DPModel import DP,DP_carb, DP_both
from Funcs_Logic_DP import logic_rollout, print_price_summary, logic_series_print
from MPC import MPC, MPC_carb, MPC_both, MPCModel
from LoadSeries import moving_average

In [2]:
house = "h16"
merged = merge(house)
merged

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001,76.0
...,...,...,...,...,...
2022-12-31 19:00:00,0.0,8.7,-8.7,498.329987,209.0
2022-12-31 20:00:00,0.0,6.9,-6.9,301.190002,224.0
2022-12-31 21:00:00,0.0,5.3,-5.3,110.730003,253.0
2022-12-31 22:00:00,0.0,5.6,-5.6,73.919998,272.0


In [5]:
battery_no_bat = Battery(max_capacity=0,max_charge=0)
type(battery_no_bat)

Battery.Battery

## Running all models

In [3]:
#"Input" variables
using_N=False
bat_capacity=13


Start="2022-06-19 00:00:00"
if using_N:
    N = 24*7+1
    End = pd.date_range(start=Start, periods=N, freq="h")[-1]

else:
    End = "2022-06-25 23:00:00"
    N=len(pd.date_range(start=Start,end=End,freq="h"))
    
remainder = N%24

if remainder==0:
    print(f"Period is of size {N} hours = {N//24} days\nPeroid from {Start} to {End}")
elif remainder==1:
    print(f"Period of size {N} hours = {N//24} days and {remainder} hour\nPeroid from {Start} to {End}")
else:
    print(f"Period of size {N} hours = {N//24} days and {remainder} hours\nPeroid from {Start} to {End}")

Period is of size 168 hours = 7 days
Peroid from 2022-06-19 00:00:00 to 2022-06-25 23:00:00


## No battery

In [4]:
battery_no_bat = Battery(max_capacity=0,max_charge=0)

series_no_battery = logic_rollout(merged.loc[Start:End].copy(), battery_no_bat)

In [5]:
print_price_summary(series_no_battery)
logic_series_print(series_no_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  802.0  DKK
Total emissions for period:  4.0  kg
Average cost per year is: 42077.0 DKK
Average emissions per year is: 233.0 kg
Number of kwh purchased in the period: 456.3
Average number of kwh purchased per year: 23951.651497005987
Average number of kwh sold per year: 34334.3748502994
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.8

  147:   1.6151,  0.1330, -4.4000, -4.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.1062,734.8425,  0.5852, 10.5955
  148:   1.5158,  0.1370, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.3053,740.1478,  0.4795, 11.0750
  149:   1.5554,  0.1540, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.2883,745.4361,  0.5236, 11.5986
  150:   1.6868,  0.1330, -2.6000, -2.6000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  4.3858,749.8219,  0.3458, 11.9444
  151:   1.9425,  0.1340, -1.7000, -1.7000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.3022,753.1241,  0.2278, 12.1722
  152:   2.0016,  0.1140, -2.9000, -2.9000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.8045,758.9286,  0.3306, 12.5028
  153:   1.6982,  0.0920,  2.4000,  2.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.4076,758.5210, -0.2208, 12.2820
  154:   1.3818,  0.0810,  8.3000,  8.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.1469,757.3741, -0.6723, 11.6097
  155:  

## Simple logic

In [6]:
battery = Battery(max_capacity=bat_capacity,max_charge=7)

series_battery = logic_rollout(merged.loc[Start:End].copy(), battery)

In [7]:
print_price_summary(series_battery)
logic_series_print(series_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  619.0  DKK
Total emissions for period:  4.0  kg
Average cost per year is: 32499.0 DKK
Average emissions per year is: 206.0 kg
Number of kwh purchased in the period: 370.1
Average number of kwh purchased per year: 19426.925748502996
Average number of kwh sold per year: 29793.90179640719
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.

  163:   2.3369,  0.1040, -1.8000,  0.0000,  0.0000, -1.8000, 13.0000, 13.0000, 11.2000, -0.0000,588.2943, -0.0000,  2.6588
  164:   2.4341,  0.1030, -5.0000,  0.0000,  0.0000, -5.0000, 11.2000, 11.2000,  6.1000, -0.0000,588.2943, -0.0000,  2.6588
  165:   2.4007,  0.0990, -6.2000, -0.1000,  0.0000, -6.1000,  6.1000,  6.1000,  0.0000,  0.2401,588.5344,  0.0099,  2.6687
  166:   2.4107,  0.0860, -6.5000, -6.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 15.6694,604.2037,  0.5590,  3.2277
  167:   2.1943,  0.1030, -6.8000, -6.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 14.9214,619.1251,  0.7004,  3.9281


## DP

In [8]:
DPbat = Battery(max_capacity=bat_capacity)
series_battery_DP_price = DP(Start,End,house,merged,deepcopy(DPbat),byday=True,ints=True,degrade=False,verbose=True)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-20 00:00:00 to 2022-06-20 23:00:00
Period from 2022-06-21 00:00:00 to 2022-06-21 23:00:00
Period from 2022-06-22 00:00:00 to 2022-06-22 23:00:00
Period from 2022-06-23 00:00:00 to 2022-06-23 23:00:00
Period from 2022-06-24 00:00:00 to 2022-06-24 23:00:00
Period from 2022-06-25 00:00:00 to 2022-06-25 23:00:00


In [9]:
print_price_summary(series_battery_DP_price)
logic_series_print(series_battery_DP_price)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  547.0  DKK
Total emissions for period:  1.0  kg
Average cost per year is: 28736.0 DKK
Average emissions per year is: 62.0 kg
Number of kwh purchased in the period: 369.8
Average number of kwh purchased per year: 19411.17844311377
Average number of kwh sold per year: 29793.901796407186
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.8

   90:   2.9634,  0.1440, -5.5000, -5.5000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000, 16.2986,266.5406,  0.7920,  7.5074
   91:   3.6528,  0.1550, -8.1000, -1.1000,  0.0000, -7.0000, 13.0000, 13.0000,  6.0000,  4.0181,270.5587,  0.1705,  7.6779
   92:   3.4436,  0.1650,-10.4000, -4.4000,  0.0000, -6.0000,  6.0000,  6.0000,  0.0000, 15.1517,285.7104,  0.7260,  8.4039
   93:   2.9704,  0.1620, -9.1000, -9.1000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, 27.0304,312.7407,  1.4742,  9.8781
   94:   2.7863,  0.1480, -8.1000, -8.1000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, 22.5687,335.3094,  1.1988, 11.0769
   95:   2.2286,  0.1380, -7.7000, -7.7000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, 17.1599,352.4693,  1.0626, 12.1395
   96:   2.1469,  0.1270, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, 11.3784,363.8477,  0.6731, 12.8126
   97:   1.9186,  0.1460, -4.5000, -4.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.6338,372.4815,  0.6570, 13.4696
   98:  

In [10]:
series_battery_DP_carb  = DP_carb(Start,End,house,merged,deepcopy(DPbat),byday=True,ints=True,degrade=False,verbose=True)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-20 00:00:00 to 2022-06-20 23:00:00
Period from 2022-06-21 00:00:00 to 2022-06-21 23:00:00
Period from 2022-06-22 00:00:00 to 2022-06-22 23:00:00
Period from 2022-06-23 00:00:00 to 2022-06-23 23:00:00
Period from 2022-06-24 00:00:00 to 2022-06-24 23:00:00
Period from 2022-06-25 00:00:00 to 2022-06-25 23:00:00


In [11]:
print_price_summary(series_battery_DP_carb)
logic_series_print(series_battery_DP_carb)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  915.0  DKK
Total emissions for period:  -9.0  kg
Average cost per year is: 48010.0 DKK
Average emissions per year is: -493.0 kg
Number of kwh purchased in the period: 558.1
Average number of kwh purchased per year: 29295.237125748503
Average number of kwh sold per year: 39677.96047904192
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000,-11.5000,  6.0000,  6.0000,  0.0000,  0.0000,  6.0000, 17.7145, 17.7145,  1.5755,  1.5755
    1:   1.4134,  0.1020, -5.3000,-12.3000,  7.0000,  7.0000,  6.0000,  6.0000, 13.0000, 17.3849, 35.0994,  1.2546,  2.8301
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000,  5.1715, 40.2709,  0.6132,  3.4433
    3:   1.0400,  0.1550, -3.8000,  2.9000,  0.0000, -6.7000, 13.0000, 13.0000,  6.3000, -0.3016, 39.9693, -0.4495,  2.9938
    4:   

  113:   2.4844,  0.1070, 16.3000, 16.3000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000, -4.0495,652.9545, -1.7441, -2.9484
  114:   2.7334,  0.1520,  8.5000, 14.5000,  0.0000, -6.0000, 13.0000, 13.0000,  7.0000, -3.9634,648.9911, -2.2040, -5.1524
  115:   3.1918,  0.1530,  2.7000,  9.7000,  0.0000, -7.0000,  7.0000,  7.0000,  0.0000, -3.0961,645.8950, -1.4841, -6.6365
  116:   3.0797,  0.1380, -0.6000, -0.6000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  1.8478,647.7428,  0.0828, -6.5537
  117:   2.6703,  0.1090, -4.7000, -4.7000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, 12.5503,660.2931,  0.5123, -6.0414
  118:   2.1749,  0.0910, -6.5000,-13.5000,  7.0000,  7.0000,  0.0000,  0.0000,  7.0000, 29.3611,689.6543,  1.2285, -4.8129
  119:   1.5650,  0.1000, -7.6000, -0.6000,  0.0000, -7.0000,  7.0000,  7.0000,  0.0000,  0.9390,690.5932,  0.0600, -4.7529
  120:   1.6726,  0.0680, -5.6000,-12.6000,  7.0000,  7.0000,  0.0000,  0.0000,  7.0000, 21.0741,711.6674,  0.8568, -3.8961
  121:  

In [12]:
series_battery_DP_both = DP_both(Start,End,house,merged,deepcopy(DPbat),byday=True,ints=True,degrade=False,verbose=True, ratio=0.5)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-20 00:00:00 to 2022-06-20 23:00:00
Period from 2022-06-21 00:00:00 to 2022-06-21 23:00:00
Period from 2022-06-22 00:00:00 to 2022-06-22 23:00:00
Period from 2022-06-23 00:00:00 to 2022-06-23 23:00:00
Period from 2022-06-24 00:00:00 to 2022-06-24 23:00:00
Period from 2022-06-25 00:00:00 to 2022-06-25 23:00:00


In [13]:
print_price_summary(series_battery_DP_both)
logic_series_print(series_battery_DP_both)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  670.0  DKK
Total emissions for period:  -7.0  kg
Average cost per year is: 35154.0 DKK
Average emissions per year is: -359.0 kg
Number of kwh purchased in the period: 438.9
Average number of kwh purchased per year: 23038.307784431134
Average number of kwh sold per year: 33421.03113772455
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000,-12.3000,  7.0000,  7.0000,  0.0000,  0.0000,  7.0000, 17.3849, 25.8571,  1.2546,  2.0081
    2:   1.2313,  0.1460, -4.2000,  0.0000,  0.0000, -4.2000,  7.0000,  7.0000,  2.8000, -0.0000, 25.8571, -0.0000,  2.0081
    3:   1.0400,  0.1550, -3.8000, -7.0000,  3.2000,  3.2000,  2.8000,  2.8000,  6.0000,  7.2798, 33.1369,  1.0850,  3.0931
    4:   

  114:   2.7334,  0.1520,  8.5000,  9.2000,  0.0000, -0.7000, 13.0000, 13.0000, 12.3000, -2.5147,489.3541, -1.3984, -2.6365
  115:   3.1918,  0.1530,  2.7000,  9.7000,  0.0000, -7.0000, 12.3000, 12.3000,  5.3000, -3.0961,486.2581, -1.4841, -4.1206
  116:   3.0797,  0.1380, -0.6000,  0.0000,  0.0000, -0.6000,  5.3000,  5.3000,  4.7000, -0.0000,486.2581, -0.0000, -4.1206
  117:   2.6703,  0.1090, -4.7000,  0.0000,  0.0000, -4.7000,  4.7000,  4.7000,  0.0000, -0.0000,486.2581, -0.0000, -4.1206
  118:   2.1749,  0.0910, -6.5000, -6.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, 14.1368,500.3949,  0.5915, -3.5291
  119:   1.5650,  0.1000, -7.6000, -7.6000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, 11.8938,512.2887,  0.7600, -2.7691
  120:   1.6726,  0.0680, -5.6000, -5.6000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  9.3663,521.6550,  0.3808, -2.3883
  121:   1.5747,  0.0690, -5.4000, -5.4000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.5035,530.1585,  0.3726, -2.0157
  122:  

## MPC

In [14]:
MPCbat = Battery(max_capacity=bat_capacity)

Mpc = MPCModel(house, 0.1, 0.0, 1, MPCbat.max_charge, MPCbat.max_capacity)
series_battery_MPC_price = MPC(Start,End,merged,deepcopy(MPCbat),Mpc,byday=True,verbose=False)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-20 00:00:00 to 2022-06-20 23:00:00
Period from 2022-06-21 00:00:00 to 2022-06-21 23:00:00
Period from 2022-06-22 00:00:00 to 2022-06-22 23:00:00
Period from 2022-06-23 00:00:00 to 2022-06-23 23:00:00
Period from 2022-06-24 00:00:00 to 2022-06-24 23:00:00
Period from 2022-06-25 00:00:00 to 2022-06-25 23:00:00


In [15]:
print_price_summary(series_battery_MPC_price)
logic_series_print(series_battery_MPC_price)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  547.0  DKK
Total emissions for period:  1.0  kg
Average cost per year is: 28728.0 DKK
Average emissions per year is: 58.0 kg
Number of kwh purchased in the period: 369.70000000000005
Average number of kwh purchased per year: 19405.929341317365
Average number of kwh sold per year: 29788.652694610784
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.496

  127:   2.4933,  0.0830, -4.2000,  0.0000,  0.0000, -4.2000,  7.3000,  7.3000,  3.0000, -0.0000,449.5212, -0.0000,  6.2590
  128:   2.5719,  0.0860, -3.1000,  0.0000,  0.0000, -3.1000,  3.0000,  3.0000,  0.0000, -0.0000,449.5212, -0.0000,  6.2590
  129:   2.4547,  0.0490,  3.7000,  3.7000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.9082,448.6130, -0.1813,  6.0777
  130:   2.2074,  0.0470,  7.6000,  7.6000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -1.6776,446.9354, -0.3572,  5.7205
  131:   1.7718,  0.0420, 12.0000, 12.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -2.1261,444.8092, -0.5040,  5.2165
  132:   1.1282,  0.0390,  9.9000,  2.9000,  0.0000,  7.0000,  0.0000,  0.0000,  7.0000, -0.3272,444.4820, -0.1131,  5.1034
  133:   1.5413,  0.0370, 20.0000, 27.0000,  0.0000, -7.0000,  7.0000,  7.0000,  0.0000, -4.1614,440.3206, -0.9990,  4.1044
  134:   1.4980,  0.0490, 18.1000, 18.1000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -2.7115,437.6092, -0.8869,  3.2175
  135:  

In [17]:
series_battery_MPC_carb = MPC_carb(Start,End,merged,deepcopy(MPCbat),Mpc,byday=True,verbose=False)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-20 00:00:00 to 2022-06-20 23:00:00
Period from 2022-06-21 00:00:00 to 2022-06-21 23:00:00
Period from 2022-06-22 00:00:00 to 2022-06-22 23:00:00
Period from 2022-06-23 00:00:00 to 2022-06-23 23:00:00
Period from 2022-06-24 00:00:00 to 2022-06-24 23:00:00
Period from 2022-06-25 00:00:00 to 2022-06-25 23:00:00


In [18]:
print_price_summary(series_battery_MPC_carb)
logic_series_print(series_battery_MPC_carb)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  913.0  DKK
Total emissions for period:  -9.0  kg
Average cost per year is: 47909.0 DKK
Average emissions per year is: -493.0 kg
Number of kwh purchased in the period: 555.2
Average number of kwh purchased per year: 29143.013173652696
Average number of kwh sold per year: 39525.736526946115
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000,-11.5000,  6.0000,  6.0000,  0.0000,  0.0000,  6.0000, 17.7145, 17.7145,  1.5755,  1.5755
    1:   1.4134,  0.1020, -5.3000,-12.3000,  7.0000,  7.0000,  6.0000,  6.0000, 13.0000, 17.3849, 35.0994,  1.2546,  2.8301
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000, 13.0000, 13.0000, 13.0000,  5.1715, 40.2709,  0.6132,  3.4433
    3:   1.0400,  0.1550, -3.8000, -0.7000,  0.0000, -3.1000, 13.0000, 13.0000,  9.9000,  0.7280, 40.9989,  0.1085,  3.5518
    4:  

  152:   2.0016,  0.1140, -2.9000, -2.9000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  5.8045,872.7005,  0.3306, -0.4942
  153:   1.6982,  0.0920,  2.4000,  2.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.4076,872.2930, -0.2208, -0.7150
  154:   1.3818,  0.0810,  8.3000,  1.3000,  0.0000,  7.0000,  0.0000,  0.0000,  7.0000, -0.1796,872.1133, -0.1053, -0.8203
  155:   1.3761,  0.0840, 10.1000, 17.1000,  0.0000, -7.0000,  7.0000,  7.0000,  0.0000, -2.3531,869.7602, -1.4364, -2.2567
  156:   1.4705,  0.0800, 17.6000, 10.6000,  0.0000,  7.0000,  0.0000,  0.0000,  7.0000, -1.5587,868.2015, -0.8480, -3.1047
  157:   1.3424,  0.1020, 19.3000, 26.3000,  0.0000, -7.0000,  7.0000,  7.0000,  0.0000, -3.5305,864.6711, -2.6826, -5.7873
  158:   1.2206,  0.0750, 18.7000, 18.7000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -2.2825,862.3886, -1.4025, -7.1898
  159:   1.3759,  0.0690, 21.3000, 14.3000,  0.0000,  7.0000,  0.0000,  0.0000,  7.0000, -1.9676,860.4210, -0.9867, -8.1765
  160:  

In [20]:
series_battery_MPC_both = MPC_both(Start,End,merged,deepcopy(MPCbat),Mpc,byday=True,verbose=False, ratio=0.5)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-20 00:00:00 to 2022-06-20 23:00:00
Period from 2022-06-21 00:00:00 to 2022-06-21 23:00:00
Period from 2022-06-22 00:00:00 to 2022-06-22 23:00:00
Period from 2022-06-23 00:00:00 to 2022-06-23 23:00:00
Period from 2022-06-24 00:00:00 to 2022-06-24 23:00:00
Period from 2022-06-25 00:00:00 to 2022-06-25 23:00:00


In [21]:
print_price_summary(series_battery_MPC_both)
logic_series_print(series_battery_MPC_both)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  618.0  DKK
Total emissions for period:  -5.0  kg
Average cost per year is: 32452.0 DKK
Average emissions per year is: -271.0 kg
Number of kwh purchased in the period: 397.0
Average number of kwh purchased per year: 20838.934131736525
Average number of kwh sold per year: 31221.657485029944
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000,-12.3000,  7.0000,  7.0000,  0.0000,  0.0000,  7.0000, 17.3849, 25.8571,  1.2546,  2.0081
    2:   1.2313,  0.1460, -4.2000,  0.0000,  0.0000, -4.2000,  7.0000,  7.0000,  2.8000, -0.0000, 25.8571, -0.0000,  2.0081
    3:   1.0400,  0.1550, -3.8000, -7.0000,  3.2000,  3.2000,  2.8000,  2.8000,  6.0000,  7.2798, 33.1369,  1.0850,  3.0931
    4:  

  144:   1.9364,  0.1300, -6.0000,-13.0000,  7.0000,  7.0000,  0.0000,  0.0000,  7.0000, 25.1728,570.0018,  1.6900,  0.6731
  145:   1.7616,  0.1460, -4.9000,  0.0000,  0.0000, -4.9000,  7.0000,  7.0000,  2.0000, -0.0000,570.0018, -0.0000,  0.6731
  146:   1.5989,  0.1550, -4.9000, -2.8000,  0.0000, -2.1000,  2.0000,  2.0000,  0.0000,  4.4769,574.4787,  0.4340,  1.1071
  147:   1.6151,  0.1330, -4.4000, -8.0000,  3.6000,  3.6000,  0.0000,  0.0000,  3.6000, 12.9204,587.3991,  1.0640,  2.1711
  148:   1.5158,  0.1370, -3.5000,-10.5000,  7.0000,  7.0000,  3.6000,  3.6000, 10.6000, 15.9159,603.3150,  1.4385,  3.6096
  149:   1.5554,  0.1540, -3.4000,  0.0000,  0.0000, -3.4000, 10.6000, 10.6000,  7.2000, -0.0000,603.3150, -0.0000,  3.6096
  150:   1.6868,  0.1330, -2.6000,  0.0000,  0.0000, -2.6000,  7.2000,  7.2000,  4.6000, -0.0000,603.3150, -0.0000,  3.6096
  151:   1.9425,  0.1340, -1.7000,  0.0000,  0.0000, -1.7000,  4.6000,  4.6000,  2.8000, -0.0000,603.3150, -0.0000,  3.6096
  152:  